In [2]:
# %% [markdown]
# # Limpieza y Normalización de Datos - Precios de Aguacate

# %%
# Importar librerías
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Cargar datos
avocado = pd.read_csv('../data/avocado.csv')

# %% [markdown]
# ## 1. Manejo de valores nulos
# %%
# Verificar valores nulos
avocado.isnull().sum()

# %% [markdown]
# ## 2. Tratamiento de outliers
# %%
# Función para identificar outliers
def detect_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] < lower_bound) | (df[column] > upper_bound)]

# Identificar outliers en volumen total
outliers = detect_outliers(avocado, 'Total Volume')
print(f"Número de outliers en Total Volume: {len(outliers)}")

# %% [markdown]
# ## 3. Transformación de variables categóricas
# %%
# Convertir tipo a dummy
avocado = pd.get_dummies(avocado, columns=['type'], drop_first=True)

# Convertir fecha a características numéricas
avocado['Date'] = pd.to_datetime(avocado['Date'])
avocado['month'] = avocado['Date'].dt.month
avocado['day_of_week'] = avocado['Date'].dt.dayofweek
avocado.drop('Date', axis=1, inplace=True)

# Codificación one-hot para region (simplificada)
top_regions = avocado['region'].value_counts().nlargest(10).index
avocado['region'] = avocado['region'].apply(lambda x: x if x in top_regions else 'Other')
avocado = pd.get_dummies(avocado, columns=['region'], drop_first=True)

# %% [markdown]
# ## 4. Normalización
# %%
# Variables a normalizar
cols_to_scale = ['Total Volume', '4046', '4225', '4770', 
                'Total Bags', 'Small Bags', 'Large Bags', 'XLarge Bags']

scaler = StandardScaler()
avocado[cols_to_scale] = scaler.fit_transform(avocado[cols_to_scale])

# %% [markdown]
# ## 5. Guardar datos limpios
# %%
avocado.to_csv('../data/avocado_clean.csv', index=False)
print("Datos limpios guardados exitosamente!")

Número de outliers en Total Volume: 2297
Datos limpios guardados exitosamente!
